In [24]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

import reverse_geocoder

In [12]:
df = pd.read_csv('20230108-icews-events.tab', sep='\t')
df.head()

,Event ID,Event Date,Source Name,Source Sectors,Source Country,Event Text,CAMEO Code,Intensity,Target Name,Target Sectors,Target Country,Story ID,Sentence Number,Publisher,City,District,Province,Country,Latitude,Longitude
0,37543936,2022-10-13,Saudi Arabia,NaN,Saudi Arabia,Deny responsibility,16,-5.0,United States,NaN,United States,56635750,4,The Oil Daily,NaN,NaN,Washington D.C.,United States,38.9171,-77.0003
1,37543653,2022-10-13,Saudi Arabia,NaN,Saudi Arabia,Praise or endorse,51,3.4,Russia,NaN,Russian Federation,56635751,2,The Oil Daily,Riyadh,NaN,Mintaqat ar Riyad,Saudi Arabia,24.6877,46.7219
2,37542178,2022-10-14,Israel,NaN,Israel,Provide aid,70,7.0,United States,NaN,United States,56633048,5,The Jerusalem Post,NaN,NaN,NaN,United States,38.8951,-77.0364
3,37542179,2022-10-14,United States,NaN,United States,Provide economic aid,71,7.4,Israel,NaN,Israel,56633048,6,The Jerusalem Post,NaN,NaN,NaN,Israel,31.7790,35.2253
4,37542131,2022-10-14,Ukraine,NaN,Ukraine,"Arrest, detain, or charge with legal action",173,-5.0,Volodymyr Zelensky,"Elite,Executive,Executive Office,Government",Ukraine,56633051,2,The Jerusalem Post,NaN,NaN,NaN,Ukraine,50.4547,30.5238


In [13]:
df = df.loc[(df['Intensity'] <= -9.0)]
conflict_countries = df['Target Country'].dropna().unique()

In [80]:
geolocator = Nominatim(user_agent="mapquestapi")


def is_conflict_country(lat, long):
    query = str(lat)+','+str(long)
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)
    res = reverse(query)
    return res.raw['address'].get('country', '')

In [81]:
# Test query
is_conflict_country(83.16666666666667, -74.20833333333334)

'Canada'

In [14]:
df = pd.read_csv('models/data/grid.csv', encoding="UTF-8")
df.head()

,lat,long,population,wind_speed,solar
0,83.166667,-74.208333,0.000009,4.253750,107.335396
1,83.166667,-74.166667,0.000011,4.253750,107.335396
2,83.166667,-74.125000,0.000011,4.253750,107.335396
3,83.166667,-74.083333,0.000011,4.253750,107.335396
4,83.166667,-74.041667,0.000008,4.266758,107.335396


In [26]:
tqdm.pandas()
test = list(df.progress_apply(lambda x: tuple([x['lat'], x['long']]), axis=1))

100%|██████████| 7550816/7550816 [01:49<00:00, 68689.48it/s]


In [15]:
coordinates = (-37.81, 144.96), (31.76, 35.21)
reverse_geocoder.search(coordinates)

Loading formatted geocoded file...


[{'lat': '-37.814',
  'lon': '144.96332',
  'name': 'Melbourne',
  'admin1': 'Victoria',
  'admin2': 'Melbourne',
  'cc': 'AU'},
 {'lat': '31.76904',
  'lon': '35.21633',
  'name': 'Jerusalem',
  'admin1': 'Jerusalem',
  'admin2': '',
  'cc': 'IL'}]

In [22]:
def get_country(coordinates):
    return reverse_geocoder.search(coordinates)[0]['cc']

In [27]:
locations = reverse_geocoder.search(test)

In [36]:
countries = [locations[i]['cc'] for i in range(len(locations))]

In [38]:
df['country'] = countries

In [39]:
df.head

<bound method NDFrame.head of                lat       long  population  wind_speed       solar country
0        83.166667 -74.208333    0.000009    4.253750  107.335396      CA
1        83.166667 -74.166667    0.000011    4.253750  107.335396      CA
2        83.166667 -74.125000    0.000011    4.253750  107.335396      CA
3        83.166667 -74.083333    0.000011    4.253750  107.335396      CA
4        83.166667 -74.041667    0.000008    4.266758  107.335396      CA
...            ...        ...         ...         ...         ...     ...
7550811 -55.916667 -67.250000    0.185715   11.827971         NaN      AR
7550812 -55.916667 -67.083333    0.040128   11.713092         NaN      AR
7550813 -55.958333 -67.333333    0.525298   11.827971         NaN      AR
7550814 -55.958333 -67.291667    1.470422   11.827971         NaN      AR
7550815 -55.958333 -67.250000    0.376293   11.827971         NaN      AR

[7550816 rows x 6 columns]>

In [41]:
df.to_csv('grid.csv',index = False)